In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle as pkl

# Estimation de π par Monte Carlo

Ecrivez une fonction tirage : float -> float x float prenant en un nombre paramètre réel m et retournant un point tiré aléatoirement selon la loi uniforme dans le carré [−m,m]×[−m,m].

In [4]:
def tirage(m):
    return random.uniform(-m,m) ,random.uniform(-m,m)

In [5]:
m=5
x=tirage(m)
print(x)

(2.7345069204982604, 1.8125048391782528)


En utilisant la fonction précédente, écrivez une fonction monteCarlo : int -> float x float np.array x float np.array prenant en paramètre N, le nombre de tirages, et retournant un triplet composé d'une estimation de π par Monte Carlo ainsi que du tableau numpy des N abscisses et du tableau des N ordonnées tirées qui ont permis d'obtenir cette estimation.

In [20]:
def monteCarlo(N):
    
    """
        int -> float x float np.array x float np.array
        N : nombre de tirages
        Retourne un triplet compose d'une estimation de pi par Monte Carlo
        ainsi que du tableau des N abscisses et du tableau des N ordonnees qui
        ont permis d'obtenir cette estimation
    """
        
    m=1
    X=[]
    Y=[]
    nb=0
    for n in range(N):
        x,y=tirage(m)
        X.append(x)
        Y.append(y)
        if x**2+y**2 <1:
            nb+=1
   

    pi = 4.0 * nb/ N
    print('pi ', pi)
    return pi,np.array(X),np.array(Y)

In [21]:
N=3
print(monteCarlo(N))

('pi ', 1.3333333333333333)
(1.3333333333333333, array([ 0.67769323,  0.3431151 , -0.96470536]), array([ 0.89955235, -0.83292618, -0.30187438]))


In [22]:
N=3
print(monteCarlo(N))

('pi ', 2.6666666666666665)
(2.6666666666666665, array([-0.93195138, -0.03095795, -0.53985766]), array([-0.88720489, -0.61559351,  0.24065675]))


Tracez le carré [−1,1]×[−1,1], le cercle de centre l'origine (0,0) et de rayon 1, et les points tirés avec une couleur différente selon qu'ils sont dans le cercle ou non. Vous pouvez utiliser le code suivant afin de faire l'affichage.

In [66]:
plt.figure()

# trace le carré
plt.plot([-1, -1, 1, 1], [-1, 1, 1, -1], '-')

# trace le cercle
x = np.linspace(-1, 1, 100)
y = np.sqrt(1- x*x)
plt.plot(x, y, 'b')
plt.plot(x, -y, 'b')

# estimation par Monte Carlo
pi, x, y = monteCarlo(int(1e4))

# trace les points dans le cercle et hors du cercle

dist = x*x + y*y
plt.plot(x[dist <=1], y[dist <=1], "go")
plt.plot(x[dist>1], y[dist>1], "ro")
plt.show()

('pi ', 3.1488)


# Décodage par la méthode de Metropolis-Hastings

La variable count est un dictionnaire : pour chaque lettre du roman War and Peace, il fournit son nombre d'occurrences dans le roman.
Le modèle de bigrammes est décrit par les variables mu et A.
La variable mu est un vecteur qui contient la distribution de probabilité initiale sur les lettres.
La variable A est une matrice qui donne les probabilités d'une lettre étant donné une autre lettre.
La variable secret contient le message à décoder.


In [24]:
(count, mu, A) = pkl.load(file("countWar.pkl", "rb"))
secret = (open("secret.txt", "r")).read()[0:-1] # -1 pour supprimer le saut de ligne
#secret


3) Les fonctions de décodage seront représentées comme un dictionnaire où la clé et la valeur stockée sont toutes deux de type caractère (une lettre est codée/décodée en une autre lettre). Ecrivez une fonction swapF : (char,char) dict -> (char,char) dict qui prend en argument une fonction de décodage τt et qui retourne une nouvelle fonction de décodage τ construite en écheangeant deux lettres c1 et c2 comme indiqué à la question 3.3 du TD:
τ(c)=τt(c) pour tout c tel que c≠c1 et c≠c2;
τ(c1)=τt(c2);
τ(c2)=τt(c1).

In [108]:
def SwapF(f):
    f2 = dict(f)#création d'un dictionnaire 
    l = len(f)
    chaine=f.keys()
    #on choisit deux lettre aléatoires 
    a = np.random.randint(0, l)
    b = np.random.randint(0, l)
    t = f2[chaine[a]]
    f2[chaine[a]] = f2[chaine[b]]
    f2[chaine[b]] = t
    return f2

In [69]:
tau = {'a' : 'b', 'b' : 'c', 'c' : 'a', 'd' : 'd' }
tau2=SwapF(tau)
tau2

{'a': 'b', 'b': 'c', 'c': 'd', 'd': 'a'}

In [70]:
def decrypt(m, f):
    u=''
    for c in m:
        u+=f[c]
    return u

 >>> decrypt ( "aabcd", tau )
 'bbcad'
 >>> decrypt ( "dcba", tau )
 'dacb'

In [71]:
print (decrypt ( "aabcd", tau ))
decrypt ( "dcba", tau )
   

bbcad


'dacb'

 Ecrivez une fonction logLikelihood : string x float np.array x float np.2D-array x char list -> string qui, étant donné un message mess (chaîne de caractères), les tableaux mu et A créés à la question 2 à partir de pickle et une liste de caractères chars, renvoie la log-vraisemblance du message mess par rapport au modèle bigramme (mu, A). Comme mu et A sont des matrices, il faut assurer la correspondance entre une lettre du message et son indice correspondant dans mu et A. Le dernier paramètre (chars) est une liste de caractères qui permet d'obtenir l'indice correspondant à une lettre, par exemple, en utilisant chars.index('a') pour récupérer l'indice de la lettre 'a'. Pour la valeur de chars, vous pourrez utiliser les clés d'un dictionnaire contenant toutes les lettres, telles que count.keys() (vous avez créé la variable count à la question 2 à partir de pickle).

In [94]:
#Le modèle de bigrammes est décrit par les variables mu et A.
#La variable mu est un vecteur qui contient la distribution de probabilité initiale sur les lettres. 
#La variable A est une matrice qui donne les probabilités d'une lettre étant donné une autre lettre. 
#La variable secret contient le message à décoder.
"""
        Le dernier paramètre (chars) est une liste de caractères qui permet d'obtenir l'indice correspondant à une lettre,
        par exemple,4 en utilisant chars.index('a') pour récupérer l'indice de la lettre 'a'. 
        Pour la valeur de chars, vous pourrez utiliser les clés d'un dictionnaire contenant toutes les lettres, telles que count.keys() 
        (vous avez créé la variable count à la question 2 à 
        partir de pickle).
       """ 
def logLikelihood(m, mu, A, chars):
    
     #renvoie la log-vraisemblance du message mess par rapport au modèle bigramme (mu, A)
        
    logL = np.log(mu[chars.index(m[0])])
    
    for i in range(len(m) - 1):
        logL += np.log(A[chars.index(m[i])] [chars.index(m[i+1])])
    return logL

p1 = logLikelihood( "abcd", mu, A, count.keys() )
p2 = logLikelihood( "dcba", mu, A, count.keys() )
print(p1)
#-24.600258560804818
print(p2)
#-26.274828997400395

-24.6002585608
-26.2748289974


# #question 6

Codez la méthode de Métropolis-Hastings vue en TD sous la forme d'une fonction appelée MetropolisHastings(mess, mu, A, tau, N) en utilisant les fonctions swapF, decrypt et logLikelihood:
Le paramètre mess est le message codé.
Les paramètres mu et A représentent le modèle de bigrammes.
L'argument tau est la fonction de décodage initiale pour démarrer l'algorithme de Métropolis-Hastings.
Enfin, l'argument N est le nombre maximum d'itérations de l'algorithme.
La méthode est une simple boucle où on fait les étapes suivantes :
tirage d'une nouvelle fonction de décodage tau' en appliquant swapF avec pour paramètre la fonction de décodage courante tau
calcul de la log-vraisemblance du message décodé grâce à tau'
tirage pour accepter ou non la transition vers tau' selon le rapport des vraisemblances
si la transition est acceptée, sauvegarder le message décodé avec la plus haute vraisemblance.
La fonction retourne le message décodé le plus vraisemblable. Vous afficherez la log-vraisemblance à chaque fois qu'elle est améliorée et le message décodé correspondant pour pouvoir observer l'évolution de l'algorithme.

"print identityTau ()\nprint identityTau ()[u'a']\nprint identityTau ().keys()"

In [74]:

def decrypt(m, f):   
    k=''
    for c in m:
        if c!='\\':
            k+=f[c]
    return k
#print (decrypt(secret ,identityTau ()))

In [96]:
def MetropolisHastings(mess, mu, A, tau, N): 
    """
        mess : message code
        mu, A : modele de bigrammes
        tau : fonction de decodage initiale
        N : nombre maximum d'iteration de l'algorithme
        Retourne le message decode le plus vraisemblable    
    """     
   
     
    oldDecode = decrypt(mess,tau) 
    oldlogV = logLikelihood(oldDecode,mu, A, count.keys() ) 
    maxDecode=oldDecode 
    maxL=oldlogV 
    for i in range(N): 
        newTau = SwapF(tau) 
        newDecode = decrypt(mess,newTau) 
        newlogV = logLikelihood(newDecode,mu, A, count.keys() ) 
        mini = min(1, newlogV/oldlogV) 
        if(np.random.uniform(0,1)<mini): 
            tau = newTau 
            oldDecode=newDecode 
            oldlogV=newlogV 
            if newlogV>maxL: 
                maxDecode = newDecode 
                maxL=newlogV 
    return maxDecode
MetropolisHastings( secret, mu, A, identityTau (), 10000)

u')bb=5MK6g=QE4xgExQ=56*x=3f6*xbx/=6b8g =3563=J6359=)ff4*4g =63=4g24g43xQ4K6bQD=K635xK634EQD=35x=K8Q3=x;6E3=82=QE4xgExQD=6L6g/8gQ=35x=Jf8ExQQ=82=6g6byQ4Q=6g/=xg3xfQ=8g=35x=gxN=Jf8ExQQ=82=35x=4g3x f6348g=82=Mg,g8NgD=4g24g43xby=QK6bbD=GM6g3434xQ9=)L6g/8g4g =35x=E8gExJ348g=82=E6MQxD=K635xK634EQ=Qxx,Q=b6ND=3563=4QD=35x=Jf8Jxf3y=E8KK8g=38=6bb=Mg,g8NgD=4g24g43xby=QK6bbD=xbxKxg3Q9=cg=6g835xf=28fK=LM3=6b8g =35x=Q6Kx=J635=82=fx2bxE348g=35x=835xf=QE4xgExQ=56*x=Jf8Exx/x/9=w5xg=(xN38g=xgMgE463x/=35x=b6N=82= f6*43y=5x=/4/=g83=Q6y=3563=35x=QMg=8f=35x=x6f35=56/=6=Jf8Jxf3y=82=633f6E348g!=5x=Q64/=3563=6bb=L8/4xQ=2f8K=35x=b6f xQ3=38=35x=QK6bbxQ3=56*x=35x=Jf8Jxf3y=82=633f6E34g =8gx=6g835xfD=3563=4QD=bx6*4g =6Q4/x=35x=GMxQ348g=82=35x=E6MQx=82=35x=K8*xKxg3=82=35x=L8/4xQD=5x=x;JfxQQx/=35x=Jf8Jxf3y=E8KK8g=38=6bb=L8/4xQ=2f8K=35x=4g24g43xby=b6f x=38=35x=4g24g43xby=QK6bb9=R5x=Q6Kx=4Q=/8gx=Ly=35x=g63Mf6b=QE4xgExQC=bx6*4g =6Q4/x=35x=GMxQ348g=82=E6MQxD=35xy=Qxx,=28f=b6NQ9=q4Q38fy=Q36g/Q=8g=35x=Q6Kx=J6359=)g/=42=54

In [102]:
def identityTau ():
    tau = {} 
    for k in count.keys ():
        tau[k] = k
    return tau


#teste  
"""print identityTau ()
print identityTau ()[u'a']
print identityTau ().keys()"""

"print identityTau ()\nprint identityTau ()[u'a']\nprint identityTau ().keys()"

In [100]:
secret2 = (open("secret2.txt", "r")).read()[0:-1]

In [101]:
MetropolisHastings( secret2, mu, A, identityTau (), 10000 )

u'D00*njm6=*LXT/=X/L*n69/*Oc69/0/3*60.=2*On6O*P6On1*DccT9T=2*6O*T=UT=TO/LTm60L4*m6On/m6OTXL4*On/*m.LO*/d6XO*.U*LXT/=X/L4*6J6=3.=L*On/*Pc.X/LL*.U*6=60qLTL*6=3*/=O/cL*.=*On/*=/W*Pc.X/LL*.U*On/*T=O/2c6OT.=*.U*j=N=.W=4*T=UT=TO/0q*Lm6004*"j6=OTOT/L1*DJ6=3.=T=2*On/*X.=X/POT.=*.U*X6jL/4*m6On/m6OTXL*L//NL*06W4*On6O*TL4*On/*Pc.P/cOq*X.mm.=*O.*600*j=N=.W=4*T=UT=TO/0q*Lm6004*/0/m/=OL1*x=*6=.On/c*U.cm*JjO*60.=2*On/*L6m/*P6On*.U*c/U0/XOT.=*On/*.On/c*LXT/=X/L*n69/*Pc.X//3/31*Cn/=*h/WO.=*/=j=XT6O/3*On/*06W*.U*2c69TOq*n/*3T3*=.O*L6q*On6O*On/*Lj=*.c*On/*/6cOn*n63*6*Pc.P/cOq*.U*6OOc6XOT.=Q*n/*L6T3*On6O*600*J.3T/L*Uc.m*On/*06c2/LO*O.*On/*Lm600/LO*n69/*On/*Pc.P/cOq*.U*6OOc6XOT=2*.=/*6=.On/c4*On6O*TL4*0/69T=2*6LT3/*On/*"j/LOT.=*.U*On/*X6jL/*.U*On/*m.9/m/=O*.U*On/*J.3T/L4*n/*/dPc/LL/3*On/*Pc.P/cOq*X.mm.=*O.*600*J.3T/L*Uc.m*On/*T=UT=TO/0q*06c2/*O.*On/*T=UT=TO/0q*Lm6001*)n/*L6m/*TL*3.=/*Jq*On/*=6Ojc60*LXT/=X/LK*0/69T=2*6LT3/*On/*"j/LOT.=*.U*X6jL/4*On/q*L//N*U.c*06WL1*STLO.cq*LO6=3L*.=*On/*L6m/*P6On1*D=3*TU*nT

In [ ]:
import numpy.random as npr

def updateOccurrences(text, count):
    for c in text:
        if c == u'\n':
            continue
        try:
            count[c] += 1
        except KeyError as e:
            count[c] = 1

def mostFrequent(count):
    bestK = []
    bestN = -1
    for k in count.keys():
        if (count[k]>bestN):
            bestK = [k]
            bestN = count[k]
        elif (count[k]==bestN):
            bestK.append(k)
    return bestK

def replaceF(f, kM, k):
    try:
        for c in f.keys():
            if f[c] == k:
                f[c] = f[kM]
                f[kM] = k
            return
    except KeyError as e:
        f[kM] = k

def mostFrequentF(message, count1, f={}):
    count = dict(count1)
    countM = {}
    updateOccurrences(message, countM)
    while len(countM) > 0:
        bestKM = mostFrequent(countM)
        bestK = mostFrequent(count)
        if len(bestKM)==1:
            kM = bestKM[0]
        else:
            kM = bestKM[npr.random_integers(0, len(bestKM)-1)]
        if len(bestK)==1:
            k = bestK[0]
        else:
            k = bestK[npr.random_integers(0, len(bestK)-1)]
        replaceF(f, kM, k) 
        countM.pop(kM)
        count.pop(k)
    return f
chars = count.keys()
tau_init = mostFrequentF(secret2, count, identityTau () )

MetropolisHastings(secret2, mu, A, tau_init, 90000 )

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:43: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:43: DeprecationWarning: This function is deprecated. Please call randint(0, 7 + 1) instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:43: DeprecationWarning: This function is deprecated. Please call randint(0, 6 + 1) instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:43: DeprecationWarning: This function is deprecated. Please call randint(0, 5 + 1) instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:43: DeprecationWarning: This function is deprecated. Please call randint(0, 4 + 1) instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:43: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:43: DeprecationWarning: Thi

In [49]:
# la probabilité du decodage etant donné le messag actuel, est un aposterio
# on part avec une vraisemblance tres petite pour avoir l ergodicité